In [3]:
import pymupdf4llm
import json
from llama_index.core.schema import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SimpleNodeParser
import pickle
import openai
from IPython.display import display, Markdown, Latex
with open('api.txt') as f:
    API_KEY = f.read().strip()
openai.api_key = API_KEY

In [ ]:
md = pymupdf4llm.to_markdown("Trading-Volatility.pdf", page_chunks=True)

In [34]:
with open("TradingVolatilityChunks.pkl", "wb") as f:
    pickle.dump(md, f)

In [2]:
llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data("Trading-Volatility.pdf")

Successfully imported LlamaIndex


In [3]:
index = VectorStoreIndex.from_documents(llama_docs)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What cause implied volatility to increase?")

In [64]:
print(response)

Implied volatility increases due to several factors. One of them is the vega convexity of options. As the vega of options rises with increasing volatility, this enlarges the size of the short volatility position that needs to be hedged. As a result, trading desks' short volatility position grows, leading them to buy more volatility to cover the increased short position, which in turn leads to further gains in implied volatility. This cycle of increasing volatility is referred to as the 'structured products vicious circle'. Another factor is the demand for put protection and OTM options, which leads to market makers raising their prices to compensate for the risk they face. This lifts the price of variance. The demand from structured products also typically lifts index implied compared to single-stock implied.


## Saving a llama index document

In [4]:
index.storage_context.persist(persist_dir="./tradingVolatilityDB")


## Local 

In [1]:
import ollama
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.llms.ollama import Ollama

In [6]:
# load from db
storageContext = StorageContext.from_defaults(persist_dir="tradingVolatilityDB")
index = load_index_from_storage(storageContext)

In [4]:
response = ollama.generate(model="deepseek-r1:7b",
                            prompt="What is implied volatility?",
                            stream=False)

KeyboardInterrupt: 

In [76]:
print(response['response'])

<think>
Okay, so I'm trying to understand why implied volatility (IV) increases. I've heard that IV is a key factor in options pricing, but I'm not entirely sure what causes it to go up or down. Let me break this down step by step.

First off, I remember reading about the Black-Scholes model, which is used to price options. It assumes that the underlying asset's price follows a log-normal distribution with constant volatility and no dividends. But in reality, markets are more unpredictable, so maybe that has something to do with it.

One thing I think might cause IV to increase is market uncertainty or fear. Like during the COVID-19 pandemic, stock markets were really volatile, and people were scared. So when there's a lot of uncertainty about the future, investors might expect bigger price swings in the underlying asset. This expectation would make the option prices more sensitive to changes, hence increasing IV.

Also, if the dividend rate increases, but the risk-free rate doesn't go

In [14]:
retriever = index.as_retriever(top_k=10)
nodes = retriever.retrieve("Who's the author of this book?")

In [15]:
display(Markdown(nodes[0].text))

###### **ACKNOWLEDGEMENTS **

I would like to thank James Peattie for persuading me to work at Merrill Lynch, and starting
my career within research which I still enjoy to this very day.

Iain Clamp aka “The Guru” deserves special recognition, for explaining the intricacies of
volatility trading.

I will always be grateful to Tom Dauterman and Irene Ferrero for their many months of
effort proofing this publication. I would also like to thank Mariano Colmenar, without
whose support this book would never have been published.

And finally, and most importantly, I would like to thank my wife Claire for her endless
patience and understanding with me while I researched and wrote this book over the past 5

years.


-----

In [18]:
deepseek = Ollama(model="deepseek-r1:7b", request_timeout=120)
queryEngine = index.as_query_engine(
    llm=deepseek,
    similarity_top_k=5,
    use_async=True,
    )

In [19]:
query = "Who's the author of this book?"
response = queryEngine.query(query)
print(response)

<think>
Okay, I need to figure out who the author is based on the provided context. Let me go through each section step by step.

First, looking at the ACKNOWLEDGEMENTS page. It says "I will always be grateful to my wife Claire for her endless patience and understanding..." which indicates that Claire is someone important in the author's life but doesn't explicitly name her as an author or a co-author.

Next, moving on to the PREFACE section. The introduction mentions that one of the main reasons for writing the book was due to the lack of other publications dealing with practical issues of using derivatives. It also says the content quickly moves onto more advanced subjects after covering introductory material. This suggests it's a standalone work by one person.

Looking through the page numbers, there are references like page 6, 241, etc., but none mention multiple authors or contributors. The information provided doesn't list any co-authors or editors, so I can assume this is a sing

In [20]:
chat_engine = index.as_chat_engine(
    llm=deepseek,
    similarity_top_k=5,
    chat_mode="context",  # this mode includes retrieval
    use_async=False
)

In [22]:
response = chat_engine.chat("Who's the author of this book?")
print(response)

<think>
Okay, so I need to figure out who the author of the book "Trading-Volatility" is based on the information provided. Let me go through each part of the query and extract any relevant details.

First, looking at the first snippet, there's an "Acknowledgements" section where James Peattie is thanked for starting the career in research. Then Iain Clamp is mentioned as someone who deserves special recognition. Tom Dauterman and Irene Ferrero are also acknowledged for their work in proofreading. Mariano Colmenar is thanked without whose support the book wouldn't have been published, which is a strong clue about the author. Lastly, Claire is mentioned with her patience and understanding.

Next, looking at the "Preamble" or preface section, it mentions that the author decided to write the book due to the lack of other publications on derivatives. It also says this publication aims to fill the gap between introductory books and more advanced ones for quantitative modeling communities. T